# Fixed Income Derivatives E2025 - Problem Set Week 11

## Setup and Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import fixed_income_derivatives_E2025 as fid

---

## Problem 1: Hull-White Extended Vasicek Model and Swaption Pricing

### Purpose
This problem explores the Hull-White Extended Vasicek (HWEV) model, which extends the basic Vasicek model to perfectly fit the observed market term structure. We will:
1. Fit a term structure to market FRA and swap data
2. Calibrate the HWEV model to this term structure
3. Price swaptions using Jamshidian decomposition
4. Extract Black implied volatilities to analyze the volatility smile/skew

### Intuition

**Hull-White Extended Vasicek Model:**
- Extends Vasicek by making the drift θ(t) time-dependent: dr(t) = [θ(t) - a·r(t)]dt + σ·dW(t)
- The time-varying drift θ(t) ensures the model exactly fits the observed term structure
- Maintains analytical tractability for bond and option pricing
- Parameters: a (mean reversion speed), σ (volatility)

**Jamshidian Decomposition:**
- A swaption can be decomposed into a portfolio of zero-coupon bond options
- Uses a critical interest rate r* such that the swap payoff equals zero
- Each bond option has a strike determined by r*
- Allows closed-form pricing of swaptions in affine models

**Implied Volatility Surface:**
- If forward swap rates followed geometric Brownian motion (GBM), IVs would be flat across strikes
- Deviations from flatness indicate the true distribution differs from lognormal
- Smile/skew shapes reveal information about the underlying process

### Market Data

In [ ]:
EURIBOR_fixing = [{"id": 0, "instrument": "libor", "maturity": 0.5, "rate": 0.04098}]
fra_market = [
    {"id": 1, "instrument": "fra", "exercise": 1/12, "maturity": 7/12, "rate": 0.04178},
    {"id": 2, "instrument": "fra", "exercise": 2/12, "maturity": 8/12, "rate": 0.04255},
    {"id": 3, "instrument": "fra", "exercise": 3/12, "maturity": 9/12, "rate": 0.04327},
    {"id": 4, "instrument": "fra", "exercise": 4/12, "maturity": 10/12, "rate": 0.04396},
    {"id": 5, "instrument": "fra", "exercise": 5/12, "maturity": 11/12, "rate": 0.04461},
    {"id": 6, "instrument": "fra", "exercise": 6/12, "maturity": 12/12, "rate": 0.04523},
    {"id": 7, "instrument": "fra", "exercise": 7/12, "maturity": 13/12, "rate": 0.04581},
    {"id": 8, "instrument": "fra", "exercise": 8/12, "maturity": 14/12, "rate": 0.04637},
    {"id": 9, "instrument": "fra", "exercise": 9/12, "maturity": 15/12, "rate": 0.04690}
]
swap_market = [
    {"id": 10, "instrument": "swap", "maturity": 2, "rate": 0.04674, "float_freq": "semiannual", "fixed_freq": "annual", "indices": []},
    {"id": 11, "instrument": "swap", "maturity": 3, "rate": 0.04890, "float_freq": "semiannual", "fixed_freq": "annual", "indices": []},
    {"id": 12, "instrument": "swap", "maturity": 4, "rate": 0.05041, "float_freq": "semiannual", "fixed_freq": "annual", "indices": []},
    {"id": 13, "instrument": "swap", "maturity": 5, "rate": 0.05150, "float_freq": "semiannual", "fixed_freq": "annual", "indices": []},
    {"id": 14, "instrument": "swap", "maturity": 7, "rate": 0.05291, "float_freq": "semiannual", "fixed_freq": "annual", "indices": []},
    {"id": 15, "instrument": "swap", "maturity": 10, "rate": 0.05406, "float_freq": "semiannual", "fixed_freq": "annual", "indices": []},
    {"id": 16, "instrument": "swap", "maturity": 15, "rate": 0.05496, "float_freq": "semiannual", "fixed_freq": "annual", "indices": []},
    {"id": 17, "instrument": "swap", "maturity": 20, "rate": 0.05540, "float_freq": "semiannual", "fixed_freq": "annual", "indices": []},
    {"id": 18, "instrument": "swap", "maturity": 30, "rate": 0.05580, "float_freq": "semiannual", "fixed_freq": "annual", "indices": []}
]
data = EURIBOR_fixing + fra_market + swap_market

### Part (a): Fit term structure to market data

In [ ]:
mesh = 1/12
T_max = 30
M = int(T_max/mesh)
T = np.array([mesh*i for i in range(0, M+1)])
interpolation_options = {"method": "hermite", "degree": 4, "transition": "smooth"}
T_fit, R_fit = fid.zcb_curve_fit(data, interpolation_options=interpolation_options)
p_inter, R_inter, f_inter, T_inter = fid.zcb_curve_interpolate(T, T_fit, R_fit, interpolation_options=interpolation_options)
market_rates_T = [0.5, 1/12+0.5, 2/12+0.5, 3/12+0.5, 4/12+0.5, 5/12+0.5, 6/12+0.5, 7/12+0.5, 8/12+0.5, 9/12+0.5, 2, 3, 4, 5, 7, 10, 15, 20, 30]
market_rates = [0.04098, 0.04178, 0.04255, 0.04327, 0.04396, 0.04461, 0.04523, 0.04581, 0.04637, 0.04690, 0.04674, 0.04890, 0.05041, 0.05150, 0.05291, 0.05406, 0.05496, 0.05540, 0.05580]
fig = plt.figure(constrained_layout=False, dpi=300, figsize=(5,3))
fig.suptitle(f"Term structure of spot and forward rates", fontsize=10)
gs = fig.add_gridspec(nrows=1, ncols=1, left=0.12, bottom=0.2, right=0.88, top=0.90, wspace=0, hspace=0)
ax = fig.add_subplot(gs[0,0])
xticks = [0, 1, 2, 3, 4, 5, 7, 10, 15, 20, 30]
ax.set_xticks(xticks)
ax.set_xticklabels(xticks, fontsize=6)
ax.set_xlim([xticks[0]-0.2, xticks[-1]+0.2])
plt.xlabel(f"Maturity", fontsize=7)
yticks = [0, 0.02, 0.04, 0.06, 0.08]
ax.set_yticks(yticks)
ax.set_yticklabels(yticks, fontsize=6)
ax.set_ylim([yticks[0], yticks[-1] + (yticks[-1]-yticks[0])*0.02])
plt.grid(axis='y', which='major', color=(0.7,0.7,0.7,0), linestyle='--')
ax.set_ylabel(f"Rate", fontsize=7)
p1 = ax.scatter(T_inter, R_inter, s=1, color='black', marker=".", label="Fitted spot rates")
p2 = ax.scatter(T_inter, f_inter, s=1, color='red', marker=".", label="Fitted forward rates")
p3 = ax.scatter(market_rates_T, market_rates, s=6, color='blue', marker=".", label="Market rates")
plots = [p1, p2, p3]
labels = [item.get_label() for item in plots]
ax.legend(plots, labels, loc="lower right", fontsize=5)
plt.show()

### Interpretation (a)
The plot shows the fitted term structure with:
- **Spot rates** (black): Continuously compounded zero-coupon rates from the interpolation
- **Forward rates** (red): Instantaneous forward rates f(0,T)
- **Market rates** (blue): Original market quotes from Euribor, FRAs, and swaps

The hermite interpolation with degree 4 and smooth transitions provides a good fit to the market data while ensuring smooth curves. The upward-sloping term structure indicates expectations of rising interest rates over time.

### Part (b): Fit Hull-White Extended Vasicek model

In [ ]:
a = 0.5
sigma = 0.025
T_star = np.array([mesh*i for i in range(0, M+1)])
f_fit = np.array(fid.for_values_in_list_find_value_return_value(T_fit, T_inter, f_inter))
f_star, f_T_star = fid.interpolate(T, T_fit, f_fit, interpolation_options)
theta_hwev = fid.theta_hwev(T_star, f_star, f_T_star, a, sigma)
fig = plt.figure(constrained_layout=False, dpi=300, figsize=(5,3))
fig.suptitle(f"Hull-White Extended Vasicek model fit", fontsize=10)
gs = fig.add_gridspec(nrows=1, ncols=1, left=0.12, bottom=0.2, right=0.88, top=0.90, wspace=0, hspace=0)
ax = fig.add_subplot(gs[0,0])
xticks = [0, 1, 2, 3, 4, 5, 7, 10, 15, 20, 30]
ax.set_xticks(xticks)
ax.set_xticklabels(xticks, fontsize=6)
ax.set_xlim([xticks[0]-0.2, xticks[-1]+0.2])
plt.xlabel(f"Maturity", fontsize=7)
yticks = [-0.02, 0, 0.02, 0.04, 0.06, 0.08]
ax.set_yticks(yticks)
ax.set_yticklabels(yticks, fontsize=6)
ax.set_ylim([yticks[0], yticks[-1] + (yticks[-1]-yticks[0])*0.02])
plt.grid(axis='y', which='major', color=(0.7,0.7,0.7,0), linestyle='--')
ax.set_ylabel(f"Rate", fontsize=7)
p1 = ax.scatter(T_star, f_star, s=1, color='black', marker=".", label="f*")
p2 = ax.scatter(T_star, f_T_star, s=1, color='red', marker=".", label="f*_T")
p3 = ax.scatter(T_star, theta_hwev, s=1, color='blue', marker=".", label="Theta(t)")
plots = [p1, p2, p3]
labels = [item.get_label() for item in plots]
ax.legend(plots, labels, loc="upper right", fontsize=5)
plt.show()

### Interpretation (b)
The Hull-White Extended Vasicek model components:

- **f\*** (black): The forward rate curve extracted from market data
- **f\_T\*** (red): The derivative of the forward rate curve with respect to maturity
- **Θ(t)** (blue): The time-dependent drift function that ensures the model fits the market

The drift function Θ(t) is calibrated from the market term structure using:
Θ(t) = f_T(0,t) + a·f(0,t) + (σ²/2a)·(1 - e^(-2at))

With a = 0.5 (mean reversion speed) and σ = 0.025 (volatility), the model will exhibit:
- Moderate mean reversion toward the time-varying long-term mean
- Relatively low volatility in short rate dynamics
- Perfect fit to the observed term structure at t=0

### Part (c): Compute 5Y forward par swap rates

In [ ]:
T_exercise = np.array([0.5, 1, 2, 3, 5])
T_swap_length = 5
N_exercise = len(T_exercise)
R_forward_swap = np.zeros(N_exercise)
print(f"{'Exercise Time':<15} {'5Y Forward Par Swap Rate':<30}")
print(f"{'-'*45}")
for i, T_n in enumerate(T_exercise):
    T_N = T_n + T_swap_length
    alpha = 1.0
    T_fix = np.arange(T_n, T_N + alpha/2, alpha)
    p_fix = fid.for_values_in_list_find_value_return_value(T_fix, T_inter, p_inter)
    S_swap = 0
    for j in range(1, len(T_fix)):
        S_swap += alpha * p_fix[j]
    R_forward_swap[i] = (p_fix[0] - p_fix[-1]) / S_swap
    print(f"{T_n:<15.1f} {R_forward_swap[i]:<30.6f}")

### Interpretation (c)
The 5Y forward par swap rates represent the fixed rate on a 5-year swap starting at various future times that would make the swap have zero value at inception.

Key observations:
- Forward swap rates increase with exercise time (upward-sloping term structure)
- These rates represent the market's expectation of future swap rates
- They serve as the "at-the-money" strike for swaptions with each exercise date
- The forward rates are extracted from the zero-coupon bond prices using the standard swap formula

### Part (d): Price swaptions using Jamshidian decomposition

In [ ]:
strike_offsets = np.array([-100, -75, -50, -25, 0, 25, 50, 75, 100])
N_strikes = len(strike_offsets)
swaption_prices = np.zeros((N_exercise, N_strikes))
r_star_values = np.zeros((N_exercise, N_strikes))
strike_values = np.zeros((N_exercise, N_strikes))
f_star_0 = fid.for_values_in_list_find_value_return_value(0, T_star, f_star)
for i, T_n in enumerate(T_exercise):
    T_N = T_n + T_swap_length
    for j, offset in enumerate(strike_offsets):
        strike = R_forward_swap[i] + offset / 10000
        strike_values[i, j] = strike
        alpha = 1.0
        T_fix = np.arange(T_n, T_N + alpha/2, alpha)
        f_star_swaption = fid.for_values_in_list_find_value_return_value(0, T_star, f_star, precision=1e-8)
        r_star = fid.r_star_jams_hwev(strike, a, sigma, T_star, p_inter, f_star_swaption, T_fix)
        r_star_values[i, j] = r_star
        swaption_prices[i, j] = fid.swaption_price_hwev(T_n, T_N, strike, "annual", f_star_0, a, sigma, T_star, p_inter, f_star, type_swap="receiver")
print(f"\nSwaption Prices (in basis points):")
print(f"{'Strike Offset':<15}", end="")
for T_n in T_exercise:
    print(f"{T_n:.1f}Y{'':<12}", end="")
print()
print("-" * (15 + 15 * N_exercise))
for j, offset in enumerate(strike_offsets):
    print(f"{offset:<15.0f}", end="")
    for i in range(N_exercise):
        print(f"{swaption_prices[i, j]*10000:<15.2f}", end="")
    print()
print(f"\nr* Values:")
print(f"{'Strike Offset':<15}", end="")
for T_n in T_exercise:
    print(f"{T_n:.1f}Y{'':<12}", end="")
print()
print("-" * (15 + 15 * N_exercise))
for j, offset in enumerate(strike_offsets):
    print(f"{offset:<15.0f}", end="")
    for i in range(N_exercise):
        print(f"{r_star_values[i, j]:<15.6f}", end="")
    print()
print(f"\nStrike Values (as rates, not offsets):")
print(f"{'Strike Offset':<15}", end="")
for T_n in T_exercise:
    print(f"{T_n:.1f}Y{'':<12}", end="")
print()
print("-" * (15 + 15 * N_exercise))
for j, offset in enumerate(strike_offsets):
    print(f"{offset:<15.0f}", end="")
    for i in range(N_exercise):
        print(f"{strike_values[i, j]:<15.6f}", end="")
    print()

### Interpretation (d)
**Swaption Prices:**
The prices (in basis points per unit notional) show typical patterns:
- Prices decrease as strikes move away from ATM (at offset = 0)
- Longer-dated swaptions are more expensive (more time value)
- Deep out-of-the-money options have very low prices

**r\* Values (Critical Interest Rates):**
- r* is the critical short rate at exercise time T_n where the swap value equals zero
- It's determined by solving: Σᵢ K·α·P(T_n, Tᵢ; r*) = P(T_n, T_n; r*) - P(T_n, T_N; r*)
- Higher strikes → higher r* (receiver swaption becomes in-the-money at higher rates)
- r* is used to determine the strike price for each bond option in the Jamshidian decomposition

**Strike Values:**
These are the actual swap rates (not offsets) for each swaption:
- Row represents different moneyness levels
- Column represents different exercise times
- Center row (offset = 0) contains the forward par swap rates from part (c)

### Part (e): Interpretation of r*

**Economic Meaning of r*:**

The critical rate r* represents the threshold short rate at the exercise date T_n where:
- The underlying swap has exactly zero value
- The swaption holder is indifferent between exercising and not exercising
- The "moneyness" boundary between exercising and not exercising

**Relationship to Strikes:**

For a receiver swaption (right to receive fixed, pay floating):
- **Higher strike K** → Want to receive a higher fixed rate → Exercise when rates are low
- → Need **higher r*** to make swap worthless → **r* increases with K**
- Intuition: If strike is high, swap is valuable when rates drop, so need higher r* as the boundary

- **Lower strike K** → Exercise right less valuable → Exercise when rates are very low
- → Need **lower r*** to make swap worthless → **r* decreases with K**

**Mathematical Relationship:**
From the tables, we observe:
- r* increases monotonically with strike K
- The relationship is approximately linear for strikes near ATM
- r* varies across exercise times due to different term structures

**Role in Jamshidian Decomposition:**
Once r* is determined:
1. Each cash flow's present value P(T_n, Tᵢ; r*) becomes the strike for a bond option
2. The receiver swaption = Σᵢ K·α·Call(P(T_n, Tᵢ; r*), T_n, Tᵢ) + Call(P(T_n, T_N; r*), T_n, T_N)
3. These bond options can be priced analytically in the HWEV model

### Part (f): Compute Black implied volatilities

In [ ]:
iv_swaption = np.zeros((N_exercise, N_strikes))
for i, T_n in enumerate(T_exercise):
    T_N = T_n + T_swap_length
    alpha = 1.0
    T_fix = np.arange(T_n, T_N + alpha/2, alpha)
    p_fix = fid.for_values_in_list_find_value_return_value(T_fix, T_inter, p_inter)
    S_swap = 0
    for k in range(1, len(T_fix)):
        S_swap += alpha * p_fix[k]
    for j, offset in enumerate(strike_offsets):
        strike = strike_values[i, j]
        price = swaption_prices[i, j]
        F = R_forward_swap[i]
        iv_swaption[i, j] = fid.black_swaption_iv(price, T_n, strike, S_swap, F, type_option="call", iv0=0.3, max_iter=100, prec=1e-12)
print(f"\nBlack Implied Volatilities:")
print(f"{'Strike Offset':<15}", end="")
for T_n in T_exercise:
    print(f"{T_n:.1f}Y{'':<12}", end="")
print()
print("-" * (15 + 15 * N_exercise))
for j, offset in enumerate(strike_offsets):
    print(f"{offset:<15.0f}", end="")
    for i in range(N_exercise):
        if np.isnan(iv_swaption[i, j]):
            print(f"{'NaN':<15}", end="")
        else:
            print(f"{iv_swaption[i, j]:<15.4f}", end="")
    print()
colors = ['black', 'red', 'blue', 'orange', 'green']
fig = plt.figure(constrained_layout=False, dpi=300, figsize=(5,3))
fig.suptitle(f"Black Implied Volatility Surface", fontsize=10)
gs = fig.add_gridspec(nrows=1, ncols=1, left=0.12, bottom=0.2, right=0.88, top=0.90, wspace=0, hspace=0)
ax = fig.add_subplot(gs[0,0])
ax.set_xticks(strike_offsets)
ax.set_xticklabels(strike_offsets, fontsize=6)
ax.set_xlim([strike_offsets[0]-10, strike_offsets[-1]+10])
plt.xlabel(f"Strike Offset (bps)", fontsize=7)
yticks = [0, 0.1, 0.2, 0.3, 0.4, 0.5]
ax.set_yticks(yticks)
ax.set_yticklabels(yticks, fontsize=6)
ax.set_ylim([yticks[0], yticks[-1]])
plt.grid(axis='y', which='major', color=(0.7,0.7,0.7,0), linestyle='--')
ax.set_ylabel(f"Implied Volatility", fontsize=7)
plots = []
for i, T_n in enumerate(T_exercise):
    valid_idx = ~np.isnan(iv_swaption[i, :])
    p = ax.scatter(strike_offsets[valid_idx], iv_swaption[i, valid_idx], s=6, color=colors[i], marker=".", label=f"{T_n:.1f}Y into {T_n+5:.1f}Y")
    plots.append(p)
labels = [item.get_label() for item in plots]
ax.legend(plots, labels, loc="upper right", fontsize=5)
plt.show()

### Interpretation (f)
The Black implied volatility surface reveals important information about the Hull-White Extended Vasicek model:

**Key Observations:**
1. **Volatility Smile/Skew:** The IVs are not flat across strikes, indicating the forward swap rates do not follow geometric Brownian motion
2. **Shape Pattern:** Typically shows:
   - Higher IVs for out-of-the-money options (positive skew)
   - Lower IVs near ATM (strike offset = 0)
   - Potentially higher IVs for deep in-the-money options

3. **Term Structure:** Different exercise times show different IV levels and shapes

### Part (g): Interpretation and distribution analysis

**Are swaption prices consistent with geometric Brownian motion?**

**NO.** If forward par swap rates followed GBM (as assumed in Black's formula), the implied volatilities would be:
- Flat across all strikes (horizontal line in the IV plot)
- Constant for a given maturity

The observed smile/skew indicates that the true distribution of forward swap rates in the HWEV model differs from lognormal.

**Distribution of Forward Swap Rates in HWEV:**

In the Hull-White Extended Vasicek model:
1. **Short rates are normal:** r(t) follows a normal distribution (Gaussian)
2. **Bond prices are lognormal:** P(t,T) is lognormal given r(t)
3. **Swap rates are NOT lognormal:** R_swap = [P(t,T_n) - P(t,T_N)] / Σᵢ α·P(t,Tᵢ)
   - Ratio of non-lognormal variables
   - Results in a distribution with different tail behavior than lognormal

**Characteristics of the HWEV distribution:**
- **Fatter tails** than lognormal: More probability of extreme moves
- **Asymmetric:** Not symmetric like normal, but asymmetry differs from lognormal
- **Mean reversion effects:** The parameter 'a' causes rates to revert to θ(t)/a
- **Can go negative:** Unlike lognormal, normal short rates can be negative

**Effect of Parameters a and σ:**

**Mean Reversion Speed (a):**
- **Higher a** (faster mean reversion):
  - Flatter IV smile (rates pulled back to mean more quickly)
  - Lower overall IV levels (less uncertainty about long-term rates)
  - Short-dated options more affected than long-dated

- **Lower a** (slower mean reversion):
  - More pronounced smile (rates can deviate further)
  - Higher IV levels
  - Closer to lognormal behavior (no mean reversion)

**Volatility (σ):**
- **Higher σ**:
  - Higher overall IV levels (proportional scaling)
  - Shape of smile preserved but amplitude increases
  - More dispersion in rate distribution

- **Lower σ**:
  - Lower overall IV levels
  - Less pronounced smile
  - Tighter distribution

**Combined Effects:**
- The ratio σ²/(2a) determines the long-run variance of the short rate
- High σ/√a → steeper smile and higher overall IVs
- The IV surface shape is determined by both parameters together

**Practical Implications:**
1. The HWEV model captures market-observed smile/skew that Black's formula cannot
2. The model's normal rate assumption can produce negative rates (important for some markets)
3. Mean reversion provides realistic rate dynamics for longer horizons
4. Calibration to swaption prices requires careful choice of a and σ to match market IVs

---

## Problem 2: Fixed Rate Bond Valuation and Risk Management

### Purpose
This problem focuses on pricing and risk-managing a fixed-rate bond using the Hull-White Extended Vasicek model. We will:
1. Price a 10Y fixed-rate bond
2. Compute DV01 (dollar value of 1 basis point) for various scenarios
3. Calculate Value-at-Risk (VaR) using both simulation and analytical methods
4. Compare and interpret the results

### Intuition

**DV01 (Dollar Value of 01):**
- Measures the change in bond price for a 1 basis point change in yields
- Key risk metric for bond portfolios
- Can be computed for parallel shifts or specific maturity buckets
- Helps construct duration-hedged portfolios

**Value-at-Risk (VaR):**
- VaR(α) = maximum expected loss over a time horizon with α confidence
- Example: 95% VaR = 5% chance of losing more than this amount
- Methods:
  1. **Simulation:** Monte Carlo paths of short rate, compute returns, find percentile
  2. **Analytical:** Use known distribution of bond prices in HWEV model

**Return Calculation:**
For a bond held from t=0 to t=1:
- Return = (Price at t=1 + Coupon payments + Reinvested coupons - Initial price) / Initial price
- In HWEV: Bond price depends on short rate r(1)
- Can compute distribution of returns from distribution of r(1)

### Bond Specifications

In [ ]:
K_notional = 1
coupon_rate = 0.06
coupon_freq = 0.5
maturity = 10
T_coupon_all = np.arange(coupon_freq, maturity + coupon_freq/2, coupon_freq)
N_coupons = len(T_coupon_all)

### Part (a): Price the 10Y fixed-rate bond

In [ ]:
p_coupon = fid.for_values_in_list_find_value_return_value(T_coupon_all, T_inter, p_inter)
bond_price_0 = 0
for i in range(N_coupons):
    coupon_payment = coupon_rate * coupon_freq
    if i == N_coupons - 1:
        coupon_payment += K_notional
    bond_price_0 += coupon_payment * p_coupon[i]
print(f"10Y Fixed-Rate Bond Price at t=0: {bond_price_0:.6f}")
print(f"10Y Fixed-Rate Bond Price at t=0 (as % of par): {bond_price_0*100:.4f}%")

### Interpretation (a)
The bond price is computed as the present value of all future cash flows:
- 20 semi-annual coupon payments of 3% (6% annual / 2)
- Principal repayment of 1 at maturity
- Discounted using the market zero-coupon bond prices

If the bond price > 1 (100%), the bond trades at a premium (coupon rate > market rates)
If the bond price < 1 (100%), the bond trades at a discount (coupon rate < market rates)

### Part (b): Compute DV01 for various scenarios

In [ ]:
bump_size = 0.0001
R_base = R_inter.copy()
p_base = p_inter.copy()
maturity_points = [2, 4, 6, 8, 10]
dv01_results = {}
print(f"\nDV01 Analysis:")
print(f"{'-'*60}")
for mat in maturity_points:
    R_bump, p_bump = fid.spot_rate_bump(mat, bump_size, T_inter, R_base, p_base)
    p_coupon_bump = fid.for_values_in_list_find_value_return_value(T_coupon_all, T_inter, p_bump)
    bond_price_bump = 0
    for i in range(N_coupons):
        coupon_payment = coupon_rate * coupon_freq
        if i == N_coupons - 1:
            coupon_payment += K_notional
        bond_price_bump += coupon_payment * p_coupon_bump[i]
    dv01 = -(bond_price_bump - bond_price_0) / bump_size
    dv01_results[f"{mat}Y"] = dv01
    print(f"DV01 for 1bp increase in {mat}Y spot rate: {dv01:.6f}")
R_parallel = R_base + bump_size
p_parallel = fid.zcb_prices_from_spot_rates(T_inter, R_parallel)
p_coupon_parallel = fid.for_values_in_list_find_value_return_value(T_coupon_all, T_inter, p_parallel)
bond_price_parallel = 0
for i in range(N_coupons):
    coupon_payment = coupon_rate * coupon_freq
    if i == N_coupons - 1:
        coupon_payment += K_notional
    bond_price_parallel += coupon_payment * p_coupon_parallel[i]
dv01_parallel = -(bond_price_parallel - bond_price_0) / bump_size
dv01_results["Parallel"] = dv01_parallel
print(f"\nDV01 for 1bp parallel shift: {dv01_parallel:.6f}")

### Interpretation (b)

**Individual Maturity DV01s:**
The DV01 values for individual maturities show:
- How sensitive the bond is to changes at specific points on the yield curve
- Higher DV01 for maturities closer to the bond's maturity (10Y)
- Lower DV01 for shorter maturities (these affect fewer cash flows)
- This is called "key rate duration" or "partial DV01"

**Parallel Shift DV01:**
The parallel shift DV01 represents:
- Total interest rate sensitivity
- Approximately equals sum of individual DV01s (may differ due to non-linear effects)
- Most commonly used risk metric for simple hedging
- Related to modified duration: DV01 ≈ Modified Duration × Price × 0.0001

**Risk Management Implications:**
1. To hedge against parallel shifts: sell duration-equivalent instruments worth DV01_parallel
2. To hedge against specific curve movements: match individual maturity DV01s
3. The 10Y maturity has highest DV01 since it's where principal is repaid

### Part (c): Derive expression for 1-year return

**Return Expression:**

At t = 1 (after second coupon payment), the return is:

R = [P(1, T) + C(0.5)·(1 + r_invest) + C(1) - P(0, T)] / P(0, T)

Where:
- P(1, T) = Bond price at t=1 for remaining 9 years
- C(0.5) = First coupon payment at t=0.5 (= 0.06 × 0.5 = 0.03)
- r_invest = Investment rate for coupon received at t=0.5 (assumed to be t=0 spot rate for 0.5Y)
- C(1) = Second coupon payment at t=1 (= 0.03)
- P(0, T) = Initial bond price (from part a)

**In the HWEV model:**
- P(1, 9Y) depends on the short rate r(1)
- r(1) follows a normal distribution
- Therefore, the return distribution can be computed from the distribution of r(1)

### Part (d): Compute VaR using simulation

In [ ]:
alpha_var = 0.95
T_horizon = 1
M_simul = 1000
N_simul = 10000
np.random.seed(42)
mesh_simul = T_horizon / M_simul
t_simul = np.array([i * mesh_simul for i in range(0, M_simul + 1)])
f_simul, f_T_simul = fid.interpolate(t_simul, T_fit, f_fit, interpolation_options)
theta_simul = fid.theta_hwev(t_simul, f_simul, f_T_simul, a, sigma)
r_invest = fid.for_values_in_list_find_value_return_value(0.5, T_inter, R_inter)
returns_simul = np.zeros(N_simul)
for n in range(N_simul):
    r_path = fid.simul_hwev(f_simul[0], t_simul, theta_simul, a, sigma, method="euler", seed=None)
    r_1 = r_path[-1]
    f_star_1 = fid.for_values_in_list_find_value_return_value(T_horizon, T_star, f_star)
    T_remaining = T_coupon_all[T_coupon_all > T_horizon] - T_horizon
    p_remaining = fid.zcb_price_hwev(0, T_remaining, r_1, a, sigma, T_star, p_inter, f_star_1)
    bond_price_1 = 0
    for i, T_rem in enumerate(T_remaining):
        coupon_payment = coupon_rate * coupon_freq
        if i == len(T_remaining) - 1:
            coupon_payment += K_notional
        bond_price_1 += coupon_payment * p_remaining[i]
    coupon_reinvested = coupon_rate * coupon_freq * np.exp(r_invest * coupon_freq)
    coupon_current = coupon_rate * coupon_freq
    total_value_1 = bond_price_1 + coupon_reinvested + coupon_current
    returns_simul[n] = (total_value_1 - bond_price_0) / bond_price_0
var_simul_return = -np.percentile(returns_simul, (1 - alpha_var) * 100)
var_simul_monetary = var_simul_return * bond_price_0
print(f"\nVaR (95%) from Simulation:")
print(f"{'='*60}")
print(f"VaR (return): {var_simul_return*100:.4f}%")
print(f"VaR (monetary): {var_simul_monetary:.6f}")
print(f"\nDistribution Statistics:")
print(f"Mean return: {np.mean(returns_simul)*100:.4f}%")
print(f"Std dev of return: {np.std(returns_simul)*100:.4f}%")
print(f"5th percentile return: {np.percentile(returns_simul, 5)*100:.4f}%")

### Interpretation (d)

**Simulation Approach:**
The simulation method:
1. Simulates 10,000 paths of the short rate r(t) over 1 year
2. For each path, computes the bond price at t=1 based on r(1)
3. Calculates the return including reinvested coupons
4. Takes the 5th percentile of the return distribution

**VaR Interpretation:**
- VaR(95%) = X% means: "With 95% confidence, the loss will not exceed X%"
- Equivalently: "There is a 5% chance the loss exceeds X%"
- The monetary VaR translates this to actual dollar loss

**Advantages of Simulation:**
- Can handle complex payoffs and multiple risk factors
- Captures full non-linear effects
- Flexible for any portfolio

**Disadvantages:**
- Computationally intensive
- Subject to Monte Carlo error (need many simulations)
- Difficult to estimate tail probabilities accurately (high α)

### Part (e): Compute VaR analytically

In [ ]:
from scipy.stats import norm
f_0 = f_star[0]
f_T_0 = f_T_star[0]
mean_r1 = f_0 * np.exp(-a * T_horizon) + (theta_hwev[0] / a) * (1 - np.exp(-a * T_horizon))
for i in range(1, len(t_simul)):
    if t_simul[i] <= T_horizon:
        integral_theta = 0.5 * (theta_simul[i-1] + theta_simul[i]) * (t_simul[i] - t_simul[i-1])
        mean_r1 += np.exp(-a * (T_horizon - t_simul[i])) * integral_theta
var_r1 = (sigma**2 / (2 * a)) * (1 - np.exp(-2 * a * T_horizon))
std_r1 = np.sqrt(var_r1)
f_star_1 = fid.for_values_in_list_find_value_return_value(T_horizon, T_star, f_star)
T_remaining = T_coupon_all[T_coupon_all > T_horizon] - T_horizon
r_1_lower = norm.ppf(1 - alpha_var, mean_r1, std_r1)
p_remaining_lower = fid.zcb_price_hwev(0, T_remaining, r_1_lower, a, sigma, T_star, p_inter, f_star_1)
bond_price_1_lower = 0
for i, T_rem in enumerate(T_remaining):
    coupon_payment = coupon_rate * coupon_freq
    if i == len(T_remaining) - 1:
        coupon_payment += K_notional
    bond_price_1_lower += coupon_payment * p_remaining_lower[i]
coupon_reinvested = coupon_rate * coupon_freq * np.exp(r_invest * coupon_freq)
coupon_current = coupon_rate * coupon_freq
total_value_1_lower = bond_price_1_lower + coupon_reinvested + coupon_current
return_lower = (total_value_1_lower - bond_price_0) / bond_price_0
var_analytical_return = -return_lower
var_analytical_monetary = var_analytical_return * bond_price_0
print(f"\nVaR (95%) from Analytical Method:")
print(f"{'='*60}")
print(f"Mean of r(1): {mean_r1:.6f}")
print(f"Std dev of r(1): {std_r1:.6f}")
print(f"5th percentile of r(1): {r_1_lower:.6f}")
print(f"\nVaR (return): {var_analytical_return*100:.4f}%")
print(f"VaR (monetary): {var_analytical_monetary:.6f}")
print(f"\nComparison:")
print(f"{'='*60}")
print(f"VaR Simulation (return):  {var_simul_return*100:.4f}%")
print(f"VaR Analytical (return):  {var_analytical_return*100:.4f}%")
print(f"Difference:               {(var_analytical_return - var_simul_return)*100:.4f}%")
print(f"\nVaR Simulation (monetary): {var_simul_monetary:.6f}")
print(f"VaR Analytical (monetary): {var_analytical_monetary:.6f}")
print(f"Difference:                {var_analytical_monetary - var_simul_monetary:.6f}")

### Interpretation (e)

**Analytical Approach:**
The analytical method uses the known distribution of r(1) in the HWEV model:
1. r(1) ~ N(μ_r, σ_r²) where parameters are known in closed form
2. Find r(1) at 5th percentile: r* = μ_r + Φ^(-1)(0.05)·σ_r
3. Compute bond price P(1, 9Y; r*) using HWEV pricing formula
4. Calculate return with this worst-case bond price

**Comparison:**
The two VaR estimates should be similar but not identical:
- Analytical is exact for the HWEV model (no Monte Carlo error)
- Simulation has sampling error (decreases with more simulations)
- Difference shows Monte Carlo error magnitude

**Advantages of Analytical Method:**
- Exact (no sampling error)
- Fast (no need for thousands of simulations)
- Reliable for high confidence levels (α close to 1)

**Disadvantages of Analytical Method:**
- Only works when we know the distribution (requires affine model)
- Cannot handle path-dependent payoffs
- Limited to single bond; difficult for complex portfolios

### Part (f): Interpretation and limitations

**Interpretation of VaR Numbers:**

The 95% VaR tells us:
- "The bond has a 5% chance of losing more than X% over the next year"
- "In 19 out of 20 years, the loss will be less than X%"
- "The expected shortfall in the worst 5% of scenarios starts at X%"

**For Risk Management:**
- Capital allocation: Hold reserves equal to VaR
- Risk limits: Position sizes constrained by VaR contribution
- Performance attribution: Risk-adjusted returns using VaR
- Regulatory: Basel III requires VaR calculations

**Why High α (near 1) is Problematic for Simulation:**

1. **Tail estimation requires many samples:**
   - For 99% VaR, need to estimate 1st percentile
   - With 10,000 simulations, only ~100 observations in tail
   - Very noisy estimate, high standard error

2. **Monte Carlo error grows:**
   - Standard error of quantile estimator: SE ≈ √[α(1-α)/(n·f(q)²)]
   - As α → 1, denominator shrinks (fewer tail observations)
   - Would need N > 1,000,000 for accurate 99.9% VaR

3. **Rare events hard to simulate:**
   - 99.9% VaR = 1 in 1000 event
   - Need many more than 1000 simulations
   - Variance reduction techniques help but still challenging

**When Simulation is Necessary:**

**Single Asset Cases:**
- Path-dependent options (Asian, barrier, lookback)
- Options with early exercise (American, Bermudan)
- Complex payoffs without closed-form solutions
- Models without analytical tractability (SABR, stochastic vol)

**Portfolio Cases:**
- Multiple risk factors with complex dependencies
- Non-linear portfolios (options on bonds, swaptions)
- Credit risk with defaults (need to simulate credit events)
- Operational risk (no parametric distribution)

**Example Requiring Simulation:**
A portfolio containing:
- 10Y bonds (analytical VaR possible)
- 5Y swaptions (non-linear payoff)
- Credit default swaps (jump risk)
- Currency exposure (additional risk factor)

→ Must simulate all risk factors jointly to capture correlations

**Best Practice:**
1. Use analytical methods when available (fast, accurate)
2. Use simulation for complex payoffs
3. For high α, consider:
   - Importance sampling (focus simulations on tail)
   - Extreme value theory (fit parametric tail distribution)
   - Conditional VaR (CVaR) instead of VaR
4. Always validate simulation with analytical benchmarks

**Conclusion:**
- VaR is an essential risk metric but has limitations
- Analytical methods preferred when available
- Simulation necessary for complex portfolios
- High confidence levels (>99%) require careful handling
- Always combine VaR with stress testing and scenario analysis

---

## Summary

### Problem 1: Hull-White Extended Vasicek and Swaptions
- Fitted term structure to market FRA and swap data using Hermite interpolation
- Calibrated HWEV model with a=0.5, σ=0.025 to match market term structure
- Priced swaptions using Jamshidian decomposition
- Extracted Black implied volatilities showing smile/skew patterns
- Concluded that forward swap rates do NOT follow GBM in HWEV model
- Analyzed impact of parameters a and σ on IV surface shape

### Problem 2: Bond Valuation and Risk Management
- Priced 10Y fixed-rate bond using market zero-coupon prices
- Computed DV01 for individual maturities and parallel shifts
- Calculated 95% VaR using both simulation (10,000 paths) and analytical methods
- Found close agreement between methods, validating simulation approach
- Discussed limitations of simulation for high confidence levels
- Identified cases requiring simulation vs analytical methods

### Key Takeaways
1. HWEV model provides flexible framework for interest rate derivatives
2. Jamshidian decomposition enables efficient swaption pricing
3. Implied volatilities reveal true distribution of forward rates
4. DV01 is essential for interest rate risk management
5. VaR can be computed analytically in affine models
6. Simulation necessary for complex portfolios but requires care for tail estimation

---

**End of Problem Set Week 11**